<a href="https://colab.research.google.com/github/natbugreal/datascience/blob/main/NataPereU9Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns
import torch
from torch import nn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#  Predicting Player Performance in Major League Baseball

Predict a player's season home run total using historical and seasonal stats.

In [15]:

baseball = pd.read_csv("https://raw.githubusercontent.com/benji555-create/Data-Science/refs/heads/main/Fixed%20data.csv").dropna()
#drops all the non-numerical columns
baseball.drop(columns=["Player","Team","Lg","Pos"],inplace=True)
baseball.reset_index(drop=True,inplace=True)

In [16]:
baseball

,k,Age,WAR,G,PA,AB,R,H,2B,3B,...,OPS,OPS+,rOBA,Rbat+,TB,GIDP,HBP,SH,SF,IBB
0,1.0,27.0,8.7,160.0,735.0,671.0,111.0,191.0,48.0,14.0,...,0.834,129.0,0.373,134.0,330.0,6.0,6.0,1.0,3.0,1.0
1,2.0,29.0,9.2,159.0,731.0,636.0,134.0,197.0,38.0,7.0,...,1.036,190.0,0.449,190.0,411.0,7.0,6.0,0.0,5.0,10.0
2,3.0,23.0,9.1,159.0,719.0,630.0,118.0,177.0,31.0,7.0,...,0.893,159.0,0.385,157.0,333.0,2.0,7.0,0.0,4.0,1.0
3,4.0,33.0,4.1,159.0,718.0,650.0,101.0,154.0,27.0,2.0,...,0.699,100.0,0.310,100.0,254.0,9.0,3.0,0.0,1.0,2.0
4,5.0,25.0,7.9,157.0,713.0,576.0,128.0,166.0,31.0,4.0,...,0.989,178.0,0.424,179.0,328.0,10.0,4.0,0.0,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,361.0,27.0,0.4,76.0,209.0,179.0,21.0,34.0,8.0,0.0,...,0.675,90.0,0.306,95.0,69.0,5.0,4.0,1.0,2.0,1.0
361,362.0,26.0,-0.5,69.0,209.0,192.0,15.0,40.0,11.0,2.0,...,0.601,65.0,0.270,64.0,64.0,2.0,3.0,0.0,1.0,0.0
362,363.0,20.0,0.1,60.0,208.0,190.0,28.0,36.0,4.0,2.0,...,0.565,66.0,0.265,70.0,59.0,1.0,2.0,0.0,1.0,0.0
363,364.0,29.0,0.9,68.0,208.0,192.0,30.0,44.0,14.0,2.0,...,0.668,93.0,0.294,87.0,77.0,3.0,1.0,2.0,3.0,0.0


In [21]:
Bat = baseball[["AB","SLG","PA"]]

In [18]:
Bat

,Rbat+,WAR,PA
0,134.0,8.7,735.0
1,190.0,9.2,731.0
2,157.0,9.1,719.0
3,100.0,4.1,718.0
4,179.0,7.9,713.0
...,...,...,...
360,95.0,0.4,209.0
361,64.0,-0.5,209.0
362,70.0,0.1,208.0
363,87.0,0.9,208.0


In [22]:
#convert to numpy array
s_matrix = Bat.to_numpy()
#convert to a PyTorch tensor
s_tensor = torch.from_numpy(s_matrix)

In [23]:
s_tensor #the default data type is a float

tensor([[6.7100e+02, 4.9200e-01, 7.3500e+02],
        [6.3600e+02, 6.4600e-01, 7.3100e+02],
        [6.3000e+02, 5.2900e-01, 7.1900e+02],
        ...,
        [1.9000e+02, 3.1100e-01, 2.0800e+02],
        [1.9200e+02, 4.0100e-01, 2.0800e+02],
        [1.9000e+02, 2.4200e-01, 2.0700e+02]], dtype=torch.float64)

In [24]:
#the dimensions of our tensor/matrix--this will be of vital importance.
s_tensor.shape

torch.Size([365, 3])

In [27]:
s_tensor

tensor([[6.7100e+02, 4.9200e-01, 7.3500e+02],
        [6.3600e+02, 6.4600e-01, 7.3100e+02],
        [6.3000e+02, 5.2900e-01, 7.1900e+02],
        ...,
        [1.9000e+02, 3.1100e-01, 2.0800e+02],
        [1.9200e+02, 4.0100e-01, 2.0800e+02],
        [1.9000e+02, 2.4200e-01, 2.0700e+02]], dtype=torch.float64)

In [34]:
from ucimlrepo import fetch_ucirepo

# Load the baseball dataset from UCI (ID: 275)
baseball = fetch_ucirepo(id=275)

# Features (X): predictor variables like hits, at-bats, etc.
x = baseball.data.features

# Targets (y): what you're trying to predict
y = baseball.data.targets

In [35]:
x


,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642
17375,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642
17376,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642
17377,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343


In [33]:
y

,cnt
0,16
1,40
2,32
3,13
4,1
...,...
17374,119
17375,89
17376,90
17377,61


In [ ]:
x_labels = ["temp", "atemp", "hum", "windspeed", "dteday", "season", "yr", "mnth", "hr", "holiday", "weekday", "workingday", "weathersit"]
y_label = "cnt"
for a in x_labels:
  plt.scatter(x[a], y[y_label])
  plt.xlabel(a)
  plt.ylabel(y)
  plt.show()